Import necessary libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import itertools


Reading the entity axes codes

In [20]:
records=[]
with open ("../data/ent_axis.features",'r') as entity_file:
    for line in entity_file:
        fields = [f.strip() for f in line.split('\t')]
        records.append(fields)
print ('Number of records: ',len(records))

Number of records:  2116531


 Building the dictionary and replacing rare causes with UNK token.

In [46]:

vocabulary_size = 4000

def build_dataset(records, vocabulary_size):
    count = [['UNK', -1]]
    causes_all=list(itertools.chain.from_iterable(records))
    count.extend(collections.Counter(causes_all).most_common(vocabulary_size - 1))
    
    dictionary = dict()
    for cause, _ in count:
        dictionary[cause] = len(dictionary)
    data = list()
    unk_count = 0
    for causes in records:
        cause_indices=[]
        for cause in causes:
            if cause in dictionary:
                index = dictionary[cause]
            else:
                index = 0  # dictionary['UNK']
                unk_count += 1
            cause_indices.append(index)
        data.append(cause_indices)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(records, vocabulary_size)
print (data[:10])
print (records[:10])
print (count[:20])
# del records  # Hint to reduce memory.
print('Most common words (+UNK)', count[:50])

record_index = 0
cause_index =0



[[6, 308], [2, 420, 14, 11, 109, 5, 4, 3], [44, 676, 44, 78, 394, 206], [21, 99, 135], [35, 104, 144, 3], [6, 50], [529, 73, 31, 66, 8, 65], [11, 12, 15, 40], [17, 6], [13, 4, 27]]
[['I500', 'L031'], ['I469', 'R042', 'C349', 'J189', 'J90', 'J449', 'I251', 'F179'], ['T71', 'X91', 'T71', 'T149', 'S223', 'S299'], ['I250', 'S720', 'X590'], ['I499', 'I516', 'E780', 'F179'], ['I500', 'I350'], ['T142', 'W19', 'R688', 'I694', 'F03', 'E039'], ['J189', 'I48', 'I64', 'G20'], ['G309', 'I500'], ['I219', 'I251', 'N19']]
[['UNK', 1999], ('I10', 357670), ('I469', 341517), ('F179', 316880), ('I251', 304988), ('J449', 262998), ('I500', 262955), ('A419', 187886), ('F03', 181293), ('J969', 168052), ('E149', 165730), ('J189', 164259), ('I48', 149003), ('I219', 133414), ('C349', 126296), ('I64', 108254), ('J960', 96845), ('G309', 91658), ('N189', 88135), ('E119', 77413)]
Most common words (+UNK) [['UNK', 1999], ('I10', 357670), ('I469', 341517), ('F179', 316880), ('I251', 304988), ('J449', 262998), ('I500',

 Generating a training batch for the skip-gram model.

In [40]:
def generate_batch(batch_size):
    global record_index
    global cause_index
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    num_added=0
    while (num_added < batch_size):
        for skip_ind in range(cause_index,len(data[record_index])):
            skip=data[record_index][skip_ind]
            for target in data[record_index]:
                if (skip == target):
                    continue

                batch[num_added]=skip
                labels[num_added,0]= target
                num_added+=1
                if (num_added >= batch_size):
                    return batch, labels
            cause_index=skip_ind
        
        record_index = (record_index + 1) % len(data)        
        cause_ind = 0
            
    
    
    return batch, labels

batch, labels = generate_batch(batch_size=20)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

6 I500 -> 308 L031
308 L031 -> 6 I500
420 R042 -> 2 I469
420 R042 -> 14 C349
420 R042 -> 11 J189
420 R042 -> 109 J90
420 R042 -> 5 J449
420 R042 -> 4 I251


Build and train a skip-gram model.

In [41]:
batch_size = 500
embedding_size = 128  # Dimension of the embedding vector.

# We pick a  validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 20     # Random set of words to evaluate similarity on.
valid_examples = np.array(range(valid_size))
num_sampled = 100    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


Begin training.

In [42]:
num_steps = 1000

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 10 == 0:
      if step > 0:
        average_loss /= 10
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  257.07220459
Nearest to UNK: W20, K858, M513, M461, I891, T175, I241, J448,
Nearest to I10: F952, T222, N251, D898, X01, T412, F320, J451,
Nearest to I469: H041, K509, M711, K613, S727, N394, Q388, I708,
Nearest to F179: T820, P281, E806, V495, K052, C160, C774, I802,
Nearest to I251: I090, C452, F458, R570, V499, W15, I311, T678,
Nearest to J449: I776, S091, Q239, M169, T839, R065, N850, R103,
Nearest to I500: S066, A165, V264, S309, J121, S024, P112, S012,
Nearest to A419: V204, A084, R221, R073, U014, G92, R488, I428,
Nearest to F03: I447, X60, P031, K413, C001, J154, C58, H540,
Nearest to J969: J152, T447, S060, Q619, S360, Y579, K226, K440,
Nearest to E149: B409, B380, S254, T845, T848, N281, Y548, G407,
Nearest to J189: T823, V204, K558, M818, X58, T802, T919, E618,
Nearest to I48: I309, A072, G933, V239, P289, W10, Y451, M609,
Nearest to I219: Q228, L859, Q211, I061, T521, T659, A491, K227,
Nearest to C349: Y836, N209, R251, C692, C700, K44

Visualize the embeddings.

In [43]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")